<center>
<h1> <b> Omdena - Crop Identification and Yield Estimation </b> </h1>
</center>

In [ ]:
#Import dependency
import ee
import geemap
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import ipywidgets as widgets
from IPython.display import display
import pickle 
import datetime 
from ipywidgets import AppLayout



In [ ]:
"""
To authenticate google earth engine.
Should have active google earth engine account.
"""
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [ ]:


#Run this cell to download the model from Omdena S3 bucket.

#!wget -O "model/sentinelModelV2.sav" "https://omdena-gpsdd-senegal.s3-us-west-1.amazonaws.com/data/Task11/trained_model_smote_sentinelV2.sav"

In [ ]:
#Global Variables
meterToAcre = (1/4047)
acreToHectare = 0.404686
cloudThreshold = 5
startDate=None
datePicker_Start=None
datePicker_End=None
roi =None
area = None
rawImg = None


filename_Sentinel = 'model/sentinelModelV2.sav'

#Classification ID and their labels
crops = {
    0:"No classification",
    1:"Cotton",
    2:"Dates",
    3:"Grass",
    4:"Lucern",
    5:"Maize",
    6:"Pecan",
    7:"Vacant land",
    8:"Vineyard",
    9:"Vineyard & Pecan"
}

In [ ]:
#User Interface Definition

#Date picker 
dateBox = widgets.DatePicker(
    description='',
    disabled=False
)

#Map
Map = geemap.Map(center=[14.607989,-14.531731], zoom=7)
Map.add_basemap('Google Satellite Hybrid')
mapWidget = widgets.Output()
with mapWidget:
    display(Map)
    
#labels
step1 = widgets.Label('Step 1: Select the date')
step2 = widgets.Label('Step 2: Select Region from the map')
date_lbl = widgets.Label('')
step2_debug = widgets.Label('')
getROI_Out_1 = widgets.Label('')
getROI_Out_2 = widgets.Label('')
getROI_Out_3 = widgets.Label('')
loadModel_Btn_Out = widgets.Label('')
step3 = widgets.Label('Step 3: Load model')
step4 = widgets.Label('Step 4: Estimate yield')
estimate_yield_debug = widgets.Label('')
yieldOutput = widgets.Label('Output:')

#Buttons
getROI = widgets.Button(description='Ok')
estimate_yield_Btn = widgets.Button(description='Estimate yield')
loadModel_Btn = widgets.Button(description='Load model')


#Progress bar
progressBar = widgets.IntProgress(
    value=0,
    min=0,
    max=19,
    step=1,
    description='',
    bar_style='info', 
    orientation='horizontal'
)

#Text Area
estimate_yield_Out = widgets.Textarea(
    value='',
    placeholder='',
    description='',
    disabled=True,
    layout={'height': '300px'}
)

#Display prediction image
estimate_yield_plot = widgets.Output()

In [ ]:


def getStatistics(change):
    """
    Collect recent satellite image from the selected date (upto 30 days before)
    The image is filtered based on cloud threshold thats been set.
    
    """
    
    global startDate, datePicker_Start, datePicker_End, roi, area,rawImg
    
    #Update UI
    step2_debug.value = 'please wait...'
    getROI_Out_1.value = ""
    getROI_Out_2.value = ""
    getROI_Out_3.value = ""
    estimate_yield_debug.value = ""
    progressBar.value = 0
    progressBar.bar_style = 'info'
    estimate_yield_Out.value = ""
    estimate_yield_Out.description=''
    
    
    #Select date range of 30 days
    startDate = dateBox.value - datetime.timedelta(days=30)
    datePicker_Start =  str(startDate.year)+"-"+str(startDate.month)+"-"+str(startDate.day)
    datePicker_End =  str(dateBox.value.year)+"-"+str(dateBox.value.month)+"-"+str(dateBox.value.day)
    
    #Get the coordinates of selected region and calculate area 
    feature = Map.draw_last_feature
    roi = feature.geometry()
    coordinate = roi.getInfo()['coordinates']
    polygon = ee.Geometry.Polygon(coordinate)
    area = polygon.area().getInfo()
    acre = area * meterToAcre
    
    #Get the lowest cloud image in last 30 days
    rawImg = Sentinel.getImage(datePicker_Start,datePicker_End,roi)
    
    #Update area information in the UI
    getROI_Out_1.value = f'Area: {area :.04f} m²'
    getROI_Out_2.value = f'Acre: {acre:.04f}'
    getROI_Out_3.value = f'Hectare: {acre * acreToHectare:.04f}' 
    
    



def plotResult(prediction):
    """
    Visualise the prediction in the form of image.
    """
    # Create colormap suitable for colourblind people and with grey as 0 for "No classification"
    cmap = sns.color_palette("colorblind", as_cmap=True)
    cmap[0], cmap[7] = cmap[7], cmap[0]

    # Plot heatmap
    ax = sns.heatmap(prediction, vmin=0, vmax=9, cmap=cmap)

    # Set colourbar
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([0.5 + (8/9*x) for x in range(10)])
    cbar.set_ticklabels(["No classification", "Cotton", "Dates", "Grass", "Lucern", "Maize", "Pecan", "Vacant land", "Vineyard", "Vineyard & Pecan"])

    # Remove axis ticks
    plt.setp(ax, yticks=[], xticks=[]);
    
    #display output
    plt.show()

    
def predictYield(predictionMatrix, crops, areaOfOnePixel):
    """
    sample value of predictionMatrix = [ [1,0,2],
                                         [0,0,1]]
    Here each value represent a pixel and the numbers represents crop ID 
    
    calculate area by multiplying total pixel count of each crop with area of 1 pixel
    """
        
    def calculateArea(cropCount):
        return cropCount*meterToAcre*areaOfOnePixel
    
    #Get pixel count of each crop
    predictionCount = np.unique(predictionMatrix, return_counts=True)
    
    #map the crop ID and total pixel count
    crop,totalcount = predictionCount[0],predictionCount[1]
    
    #Intialize the final output as string
    res=""
    for cropId,cropCount in zip(crop,totalcount):
        #Replace crop ID with crop label and update its area 
        res = res + f'{crops[cropId]} : {calculateArea(cropCount):.04f} acre\n'
        #present the output in UI
        estimate_yield_Out.value = res



class Sentinel:
    def __init__(self):
        self.model = None
        self.prediction = None
        self.imageDate = None
       
    def getImage(datePicker_Start, datePicker_End, roi, cloud=cloudThreshold):
        """
        Collect the image from sentinel satellite for the
        specified date and region
        """

        def maskS2clouds(image):
            """
            To remove cloud from the image
            """
            qa = image.select('QA60')
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                     qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000) 

        # Load Sentinel-2 TOA reflectance data.
        imageCollection = ee.ImageCollection('COPERNICUS/S2') \
            .filterDate(datePicker_Start, datePicker_End) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud)) \
            .filterBounds(roi) \
            .sort('CLOUDY_PIXEL_PERCENTAGE')


        count = imageCollection.size().getInfo()
        
        #If there is a image for the specified date then proceed
        if count<=0:
            #update UI 
            step2_debug.value = f"No images found for the date range.Choose different date"
            estimate_yield_debug.value = ""
            return None
        #Update UI
        step2_debug.value = f"Total satellite images available across the date range: {count}"
        
        #Get the lowest cloud image
        rawImgDate = ee.Image(imageCollection.first()).date()
        #Update UI with the date of the image
        step2_debug.value = f"Available date: {rawImgDate.format('yyyy-MM-dd').getInfo()}"
        #Remove cloud from the image
        rawImg = imageCollection.map(maskS2clouds).first()
        
        #visualize the selected region in the map
        visParams = {"bands": ['B4', 'B3', 'B2'],"min": 0.0, "max": 0.3};
        Map.centerObject(roi,zoom=16);
        Map.addLayer(rawImg.clip(roi), visParams, 'RGB');
        
        #return the collected image
        return rawImg

    
    
    
    def createMatrix(rawImg,roi):
        """
        To convert the satellite image into matrix form.
        """
        #Arrange the band order as same as training order
        bands = ['B1','B10','B11','B12','B2','B3','B4','B5','B6','B7','B8','B8A','B9','NDVI']
        #intialize the matrix for each band
        matrix = {'B1':None,
                  'B10':None,
                  'B11':None,
                  'B12':None,
                  'B2':None,
                  'B3':None,
                  'B4':None,
                  'B5':None,
                  'B6':None,
                  'B7':None,
                  'B8':None,
                  'B8A':None,
                  'B9':None,
                  'NDVI':None
                 }
        #set B2 band as target shape
        Res_ten= rawImg.select(['B2'])
        #convert image in to numpy array
        Res_ten = geemap.ee_to_numpy(Res_ten, region=roi)
        #get the target size
        targetSize = (Res_ten.shape[1],Res_ten.shape[0])
        
        
        for band in bands:
            #Update progress bar
            progressBar.value = progressBar.value +1
            #Update UI
            estimate_yield_debug.value = f"Processing: {band}"
            
            #calculate NDVI and update matrix
            if band == "NDVI":
                NIR = matrix['B8']
                RED = matrix['B4']
                NDVI_val = (NIR-RED)/(NIR+RED)
                matrix[band] = NDVI_val
            #update matrix 
            else:
                temp = rawImg.select([band])
                temp = geemap.ee_to_numpy(temp, region=roi)
                #resize image to target size
                matrix[band] = cv2.resize(temp,targetSize,interpolation=cv2.INTER_NEAREST)      
                
        #stack the matrix in a way to get all band information for a pixel
        FinalImage = np.dstack((matrix['B1'],matrix['B10'],matrix['B11'],
                               matrix['B12'],matrix['B2'],matrix['B3'],
                               matrix['B4'],matrix['B5'],matrix['B6'],
                               matrix['B7'],matrix['B8'],matrix['B8A'],
                               matrix['B9'],matrix['NDVI']))
        
        #return the matrix
        return FinalImage
    
    def calculate_one_pixel_area(matrix):        
        TotalPixel = matrix.shape[0]*matrix.shape[1]
        #update UI
        estimate_yield_debug.value = f"Total Pixel: {TotalPixel}"
        areaOfOnePixel = area/TotalPixel
        estimate_yield_debug.value = f"Area of one pixel: {areaOfOnePixel:.04f} Acre"
        estimate_yield_debug.value = f"Total area: {area:.04f} m²"
        return areaOfOnePixel
    
    def loadModel(self,filename_Sentinel):
        """
        load the Random forest model
        """
        self.model = pickle.load(open(filename_Sentinel, 'rb'))
        
    def predictCrop(self, inputBands):
        """
        get the probability of each crop.
        sum of all the crops equals to one.
        """
        return self.model.predict_proba([inputBands])
    
    def initPrediction(self, row, col):
        """
        initialize prediction matrix.
        shape of the matrix is same as the collected image.
        """
        self.prediction = np.zeros([row,col])
        
    #update prediction matrix only if the probability crosses the threshold
    def updateMatrix(self,inputBands,row,col,threshold =0.6):
        pred = self.predictCrop(inputBands)
        if pred.max() >= threshold:
            self.prediction[row][col] = pred.argmax()+1  
            # Add 1 to keep "no crop classified" as 0

def loadModel(change):   
    #load the model
    loadModel_Btn_Out.value = f"Loading model..."
    sentinel.loadModel(filename_Sentinel)
    loadModel_Btn_Out.value = f"Finished loading model" 

def estimateYield(change):
    progressBar.value = 0
    estimate_yield_plot.clear_output()
    
    #If there is no error in collecting the data from earth engine then proceed
    if rawImg:
        estimate_yield_debug.value = f"Collecting data"
        progressBar.value = progressBar.value +1
        
        #convert collected image in the form of matrix
        Img = Sentinel.createMatrix(rawImg,roi)
        
        estimate_yield_debug.value = f"Data collected"
        progressBar.value = progressBar.value +1
        
        #calculate area of one pixel
        areaOfOnePixel = Sentinel.calculate_one_pixel_area(Img)

        #Initialize prediction matrix
        sentinel.initPrediction(Img.shape[0],Img.shape[1])
        
        estimate_yield_debug.value = f"Inferencing the model. Please Wait..."
        progressBar.value = progressBar.value +1
        
        #loop through each pixel
        for row in range(Img.shape[0]):
            for col in range(Img.shape[1]):
                #Inference the model
                sentinel.updateMatrix(Img[row][col], row, col)
        
        #Update UI
        estimate_yield_debug.value = f"Finished prediction"
        progressBar.bar_style = "success"       
        progressBar.value = progressBar.value +1
        predictYield(sentinel.prediction, crops, areaOfOnePixel)
        progressBar.value = progressBar.value +1
        
        #Visualize the prediction
        with estimate_yield_plot:
            plotResult(sentinel.prediction)
    else:
        estimate_yield_debug.value = f"Error"
        progressBar.bar_style = "danger" 

In [ ]:
#Intialize the class
sentinel = Sentinel()

#Intialize buttons
getROI.on_click(getStatistics)
loadModel_Btn.on_click(loadModel)
estimate_yield_Btn.on_click(estimateYield)

In [ ]:
#Arrange the layout
vBox = mapWidget
vBox1 = widgets.VBox([step1, dateBox,vBox])
vBox2 = widgets.VBox([step2, getROI,step2_debug,getROI_Out_1,getROI_Out_2,getROI_Out_3,
                      step3,loadModel_Btn,loadModel_Btn_Out,
                      step4,estimate_yield_Btn,progressBar,estimate_yield_debug,yieldOutput,
                      estimate_yield_Out,estimate_yield_plot])

AppLayout(header=None,
          left_sidebar=vBox1,
          right_sidebar=vBox2,
          footer=None,
          height="70%", width="150%")